# 目的
`SQLBaseforFX`クラスによって基本デーブル`main`が作成され、そのテーブルに日付とレートが記録される。   
`SQLAnaforFX`によって解析データのテーブル`askXXmin`および`bidXXmin`が作成され、それらのテーブルにOHLCやSMAなどの解析結果が保存される。   
現状では休日に対してもOHLCが計算されているが、実データがないために休日の部分はすべてゼロとして保存される。これは無駄な部分である（特にグラフにする際に）。   
そこで`is_weekend`メソッドによってOHLCの保存方法を変える。つまり土日を除いたOHLCデータ保存に切り替える。   
その次に、データベースから呼び出したデータに対してSMAなどを計算するクラスを用意する。   

SMAなどは、OHLCが計算されたテーブルに後から追加されるため、alterクエリとupdateクエリを用いて挿入していくことになる。   
リアルタイム解析の場合は、OHLCなどと同時に挿入されるかもしれない（そちらの方が挿入にかかる時間が省ける）。   

In [1]:
import datetime
from FX.FX import SQLBaseforFX, SQLAnaforFX, OHLCanalyzer, datetimefuncs

## SQLAnaforFXのテスト

In [2]:
sql = SQLAnaforFX(year_month="201704")
# sql.initialize_ana_tables(True)

In [3]:
# sql.addOHLCRecord((datetime.datetime.now(), 111, 112, 111, 111), 15, "ask")
# res = sql.execute("delete from ask15min where datetime = '2017-05-01 20:00:49.028145'")
sql.toDataFrame(tblname="ask15min")

,open,high,low,close
2017-04-01 01:07:13,111.43,111.49,111.40,111.47
2017-04-01 01:22:13,111.47,111.49,111.44,111.46
2017-04-01 01:37:13,111.47,111.47,111.39,111.43
2017-04-01 01:52:13,111.43,111.43,111.29,111.32
2017-04-01 02:07:13,111.32,111.34,111.28,111.33
2017-04-01 02:22:13,111.33,111.41,111.33,111.40
2017-04-01 02:37:13,111.40,111.45,111.39,111.44
2017-04-01 02:52:13,111.44,111.45,111.37,111.40
2017-04-01 03:07:13,111.40,111.44,111.35,111.37
2017-04-01 03:22:13,111.37,111.39,111.25,111.26


## `is_weekend`のテスト

In [4]:
datetimefuncs.is_weekend(datetime.datetime.now()), datetimefuncs.is_weekend(datetime.datetime.now() + datetime.timedelta(0, 11*3600))

(False, None)

In [5]:
import os
os.path.exists("../SQLtest/")

True

## データベース用クラスへのSMA用メソッドの追加

In [2]:
class SQLAnaOfSMA(SQLAnaforFX):
    """
    ノートブック上の拡張クラス。
    実装された内容は`SQLAnaforFX`または`SQLBaseforFX`に移行する。
    """
    def __init__(self, *args, **kwargs):
        """
        Initialization
        """
        super().__init__(*args, **kwargs)
    
    def addSMARecordFromDataFrame(self, df, minute, bid_or_ask):
        """
        Add records realted to SMA.
        """
        datetimes = df.index
        colnames = df.columns
        tblname = self._select_ana_table(minute, bid_or_ask)
        
        res = self.execute("PRAGMA table_info({})".format(tblname))
        current_colnames = [row[1] for row in res]
        
        # Check the existence of columns named `colname` in `colnames`. 
        for colname in colnames:
            if colname not in current_colnames:
                _ = self.addcolumn(tblname, colname, "real")
        
        # Update the table.
        data = df.as_matrix()
        setcols = ",".join(["{0} = ?".format(colname) for colname in colnames])
        update_sql = "update {0} set {1} where datetime = ?".format(tblname, setcols)
        dataset = [None]*len(datetimes)
        for ii in range(len(datetimes)):
            _ = list(data[ii])
            _.append(datetimes[ii])
            dataset[ii] = tuple(_)
        self.executemany(update_sql, dataset)

## SMAanalyzer
SMAを計算するクラス。   
現時点ではOHLCanayzerと別のクラスとして用意しておくが、、

In [8]:
from FX.FX import analyzefuncs as af
class FXanalyzerSMA(object):
    def __init__(self):
        self._sma_size = [7, 13, 25]
    
    def calc_sma(self, df):
        """
        Calculate SMA trends.
        datetimes = df.index
        close = df["close"]
        """
        self.datetimes = df.index
        close_data = df["close"].as_matrix()
        self.sma_dict = af.get_sma(close_data, self._sma_size)
    
    def toDataFrame(self):
        colnames = ["sma{0:02d}".format(n) for n in self._sma_size]
        _ = [self.sma_dict[key] for key in colnames]
        data = np.zeros((len(colnames), len(_[0])))
        for ii in range(len(_)):
            data[ii] = _[ii]
        df = pd.DataFrame(data.transpose(), columns=colnames)
        df.index = self.datetimes
        return df

### calc\_SMAの実行

In [23]:
minute = 5
bid_or_ask = "ask"

In [24]:
sql = SQLAnaOfSMA(year_month="201704")
df = sql.AnaToDataFrame(minute=minute, bid_or_ask=bid_or_ask, colselect=["close"])

ana = FXanalyzerSMA()
ana.calc_sma(df)

In [26]:
df = ana.toDataFrame()
sql.addSMARecordFromDataFrame(df, minute=minute, bid_or_ask=bid_or_ask)

In [6]:
minute = 5
bid_or_ask = "ask"
sql.AnaToDataFrame(minute=1, bid_or_ask=bid_or_ask)

,open,high,low,close
2017-04-01 00:53:13,111.43,111.43,111.40,111.42
2017-04-01 00:54:13,111.43,111.44,111.41,111.41
2017-04-01 00:55:13,111.41,111.43,111.40,111.43
2017-04-01 00:56:13,111.43,111.45,111.43,111.45
2017-04-01 00:57:13,111.45,111.45,111.43,111.44
2017-04-01 00:58:13,111.44,111.44,111.43,111.43
2017-04-01 00:59:13,111.42,111.43,111.42,111.43
2017-04-01 01:00:13,111.43,111.43,111.40,111.43
2017-04-01 01:01:13,111.43,111.44,111.43,111.43
2017-04-01 01:02:13,111.43,111.44,111.43,111.43


## 全解析テーブルに対してSMAを計算

In [10]:
year_month = "201704"

sql = SQLAnaOfSMA(year_month=year_month)
ana = FXanalyzerSMA()

for bid_or_ask in ["ask", "bid"]:
    for minute in sql._mins:
        df = sql.AnaToDataFrame(minute=minute, bid_or_ask=bid_or_ask, colselect=["close"])
        ana.calc_sma(df)
        df = ana.toDataFrame()
        sql.addSMARecordFromDataFrame(df, minute=minute, bid_or_ask=bid_or_ask)

## テストコード

In [ ]:
",".join(["open", "high", "low", "close"])

In [ ]:
df = pd.DataFrame(np.array([[1,2,3,4]]), columns=["open", "high", "low", "close"])
colnames = list(df.columns)
",".join(colnames)

In [25]:
def get_sma(close_data, sma_size=[7, 13, 25]):
    """
    Simple Moving Average (SMA)を計算する
    ＜入力＞
        close_data: 終値のデータ列
        sma_size: SMAのサイズ
    ＜出力＞
        sma_dict: SMA dictionary object
    """
    if type(close_data) not in [list, np.ndarray]:
        raise TypeError
    elif type(close_data) == list:
        data = np.array(close_data)
    else:
        data = close_data.copy()

    sma_dict = dict()
    for size in sma_size:
        key = "sma{0:02d}".format(size)
        sma = [len(data[max(0, ii-size+1):ii+1]) for ii in range(len(data))]
        sma_dict[key] = sma
    return sma_dict

In [21]:
a = np.arange(0, 10)
a, af.get_sma(a)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 {'sma07': [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.5, 5.5],
  'sma13': [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5],
  'sma25': [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5]})

In [28]:
a = np.arange(0, 15)
get_sma(a)

{'sma07': [1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7],
 'sma13': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 13],
 'sma25': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}

In [3]:
sql = SQLAnaOfSMA(year_month="201704")
sql.addcolumn("ask01min", "tes", "real")
sql.execute("PRAGMA table_info({})".format("ask01min"))

[(0, 'datetime', 'varchar(255)', 0, None, 0),
 (1, 'open', 'real', 0, None, 0),
 (2, 'high', 'real', 0, None, 0),
 (3, 'low', 'real', 0, None, 0),
 (4, 'close', 'real', 0, None, 0),
 (5, 'tes', 'real', 0, None, 0)]

In [4]:
sql.deletecolumn("ask01min", "tes")
sql.execute("PRAGMA table_info({})".format("ask01min"))

[(0, 'datetime', 'varchar(255)', 0, None, 0),
 (1, 'open', 'real', 0, None, 0),
 (2, 'high', 'real', 0, None, 0),
 (3, 'low', 'real', 0, None, 0),
 (4, 'close', 'real', 0, None, 0)]

In [5]:
sql.AnaToDataFrame(minute=1, bid_or_ask=bid_or_ask)

NameError: name 'bid_or_ask' is not defined